In [ ]:
#default_exp labeller.metrics

In [ ]:
#export
from fastai2.basics import *
from pigboat.basics import *
from pigboat.labeller.core import Labeller

# Labeller metrics
> Utility metrics like `Covereage`

In [ ]:
#export
class Coverage:
    def reset(self): self.total,self.cov = 0,None
    
    def accumulate(self, b):
        #TODO: Hardcoded 0 for abstain, can be wrong
        self.total += b.shape[0]
        bcov = (b!=0).sum(axis=0)
        self.cov = bcov if self.cov is None else self.cov + bcov
        
    @property
    def value(self): return (self.cov.float()/self.total).tolist()
    @property
    def name(self): return self.__class__.__name__

In [ ]:
#export
# Remove abstain from polarity
class Polarity:
    def reset(self): self._unique = None
    
    @property
    def unique(self): 
        unique = self._unique.copy()
        unique.map(self._discard_abstain)
        return unique
        
    def accumulate(self, b):
        bpol = L(set(t.unique().tolist()) for t in b.unbind(dim=1))
        if self._unique is None: self._unique = bpol
        else: 
            for i in range_of(bpol): self._unique[i].update(bpol[i])
    
    def _discard_abstain(self, o): return o.discard(0) #TODO: Hardcoded 0
    
    @property
    def value(self): return self.unique.map(len)
    @property
    def name(self): return self.__class__.__name__

In [ ]:
#export
defaults.labeller_metrics = [Coverage, Polarity]

In [ ]:
#export
_old_labeller_init = Labeller.__init__
@patch
def __init__(self:Labeller, abstain='abstain', metrics=None):
    _old_labeller_init(self, abstain=abstain)
    self.metrics = L(instantiate(o) for o in L(metrics)+L(defaults.labeller_metrics))

In [ ]:
#export
@patch
def summary(self:Labeller, dl):
    for metric in self.metrics: metric.reset()
    for b in dl:
        for metric in self.metrics: metric.accumulate(b)
    data = dict(self.metrics.map(lambda o: (o.name, o.value)))
    return pd.DataFrame(data, index=self.func_order)

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.transforms.ipynb.
Converted 02_labeller.core.ipynb.
Converted 02a_labeller.metrics.ipynb.
Converted 03_model.majority_label_voter.ipynb.
Converted 05_text.core.ipynb.
Converted 06_text.labellers.ipynb.
Converted Untitled-Copy1.ipynb.
Converted index.ipynb.
Converted resume-Copy1.ipynb.
Converted resume.ipynb.
Converted rx_transform.ipynb.
Converted rx_transform2-Copy1.ipynb.
Converted rx_transform2.ipynb.
